# Lagrange implementation of Chebyshev nodes

In [ ]:
using Plots

In [ ]:
#implementation cell
#returns wj assuming first kind Chebyshev nodes 
#needs j, the weights' index, and n , the nodes' number.
function wj_unif(j::Int, n::Int)
    return (-1)^(j+1)*factorial(n)/(factorial(j)*factorial(n-j))
end

function wj_cheb_1(j::Int, n::Int)
    return (-1)^j*sin(pi*(2j-1)/2n)
end

#returns wj assuming second kind Chebyshev nodes 
#needs j, the weights' index, and n , the nodes' number.
function wj_cheb_2(j::Int, n::Int)
    if j==1 || j==n
        return 0.5(-1)^j
    else 
        return (-1)^j
    end
end

#returns p(x), including extrema and using Chebyslev points.
#Requires n number of nodes, (a,b) interval extrema, f the exact function, pt_type type of points
function lag_fit(n::Int, a::Number, b::Number, f::Function, pt_type::String)
    if a>b
        c=a
        a=b
        b=c        
    end
    #Case of uniform points 
    if pt_type == "unif"
        xn = [a + (j-1)*(b-a)/(n-1) for j in 1:1:n]
        weights = [wj_unif(j, n) for j in 1:1:n]
    #Case of first kind Chebyshev points (endpoints escluded)
    elseif pt_type == "c1"
        #change of coordinates inside xn declaration
        xn = [a + (b-a)*((cos((2j-1)pi/(2n)))+1)/2 for j in 1:1:n]
        weights = [wj_cheb_1(j, n) for j in 1:1:n]
    #Case of second kind Chebyshev points (including endpoints)
    elseif pt_type == "c2"
        #change of coordinates inside xn declaration
        xn = [a + (b-a)*((cos((j-1)pi/(n-1)))+1)/2 for j in 1:1:n]
        weights = [wj_cheb_2(j, n) for j in 1:1:n]
    else
        throw(DomainError(pt_type, "Non valid pt_type. Must be 'unif' or 'c1' or 'c2'"))
    end

    yn = f.(xn) 
    
    p = function(x)
        num = 0.0
        den = 0.0
        for j in 1:1:n
            if x != xn[j]
                num += weights[j]*yn[j]/(x-xn[j])
                den += weights[j]/(x-xn[j])
            else
                return yn[j]
            end
        end
        return num/den
    end

    return p, xn, yn
end

In [ ]:
n = 6
a = -1
b = 5
f(x) = exp(x)
cheb = "c2"
q, xn, yn= lag_fit(n, a, b, f, cheb)

In [ ]:
x = [i for i in a:0.01:b]
plot(x, q.(x),
     label = "fit")

plot!(x, f.(x),
     label="exact function")
scatter!(xn, yn)